# Imports

In [2]:
import pandas as pd
import numpy as np

# Load Data

In [34]:
y = pd.read_csv('data/train.csv')
y

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [ ]:
def update_dataset(dataset, signs, col_names):
    print(f'recordings found: {y.loc[[s in signs for s in y.sign]].shape[0]}')
    for i, sign in enumerate(signs):
        print(f'sign: {sign}, {i+1}/{len(signs)}')
        for j, participant_id in enumerate(y.participant_id.unique()):
            print(f'    pid: {j+1}/21')
            y_subset = y.loc[(y.sign == sign) & (y.participant_id == participant_id)]
            data = []
            n = y_subset.shape[0]
            for k, path in enumerate(y_subset.path):
                print(f'        recording: {k+1}/{n}')
                path_D = "D:/" + path
                df = pd.read_parquet(path_D)
                this_data = []
                for type in set(df.type):
                    df_t = df.loc[df.type == type]
                    for lm_i in set(df_t.landmark_index):
                        df_t_i = df_t.loc[(df_t['x'].notna()) & (df_t['y'].notna()) & (df_t.landmark_index == lm_i)]
                        if df_t_i.shape[0] != 0:
                            this_data.append(df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[0]['y'])
                            this_data.append(df_t_i.iloc[-1]['x'] - df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[-1]['y'] - df_t_i.iloc[0]['y'])
                        else:
                            this_data.extend([0,0,0,0])
                data.append(this_data + [sign])
            sign_df = pd.DataFrame(data, columns=col_names)
            dataset = pd.concat((dataset,sign_df), axis=0, ignore_index=True)
    return dataset

#### Signs used on round two of data collection
['pig',
 'elephant',
 'mad',
 'finger',
 'green',
 'up',
 'bad',
 'finger',
 'bye',
 'airplane',
 'mom',
 'kitty',
 'home',
 'yellow',
 'shirt',
 'talk',
 'icecream',
 'green',
 'clown',
 'lion',
 'bad',
 'thirsty',
 'elephant',
 'book',
 'shirt',
 'dog',
 'finger',
 'pretty',
 'vacuum',
 'vacuum',
 'chocolate',
 'pretty',
 'same',
 'red',
 'every',
 'feet',
 'drawer',
 'gum',
 'drop',
 'story',
 'puppy',
 'haveto',
 'book',
 'bad',
 'zebra',
 'cat',
 'cow',
 'mad',
 'dryer',
 'old',
 'clown',
 'garbage',
 'stairs',
 'cute',
 'wolf',
 'for',
 'tree',
 'stairs',
 'sick',
 'finish',
 'bird',
 'nose',
 'no',
 'drop',
 'yellow',
 'garbage',
 'have',
 'blow',
 'wet',
 'airplane',
 'vacuum',
 'no',
 'bye',
 'nose',
 'give',
 'puppy',
 'gum',
 'dry',
 'shower',
 'another',
 'every',
 'haveto',
 'wolf',
 'bee',
 'empty',
 'puppy',
 'thirsty',
 'night',
 'gum',
 'sick',
 'drawer',
 'nose',
 'have',
 'pretty',
 'airplane',
 'cry',
 'tiger',
 'mom',
 'talk',
 'have',
 'green',
 'zebra',
 'because',
 'gum',
 'puppy',
 'mad',
 'wolf',
 'donkey',
 'yes',
 'yucky',
 'why',
 'no',
 'tree',
 'tree',
 'dryer',
 'morning',
 'that',
 'orange',
 'where',
 'wolf',
 'tomorrow',
 'tomorrow',
 'gum',
 'uncle',
 'shower',
 'mouth',
 'nose',
 'farm',
 'frog',
 'table',
 'green',
 'another',
 'helicopter',
 'talk',
 'dry',
 'hear',
 'yourself',
 'cute',
 'gum',
 'garbage',
 'dryer',
 'find',
 'kitty',
 'chocolate',
 'no',
 'bird',
 'girl',
 'story',
 'red',
 'cry',
 'story',
 'yourself',
 'vacuum',
 'green',
 'another']

In [24]:
col_names = pd.read_csv('data/col_names.csv', index_col=0)['0']
dataset = pd.DataFrame([],columns=col_names)
signs = ['hungry']

dataset = update_dataset(dataset,signs,col_names)


recordings found: 1
sign: hungry, 1/1
    pid: 1/21
        recording: 1/1


# Load dataset to CSV

In [30]:
dataset

0,px_0_face,py_0_face,dx_0_face,dy_0_face,px_1_face,py_1_face,dx_1_face,dy_1_face,px_2_face,py_2_face,...,dy_30_pose,px_31_pose,py_31_pose,dx_31_pose,dy_31_pose,px_32_pose,py_32_pose,dx_32_pose,dy_32_pose,sign
0,0,0,0,0,0,0,0,0,0,0,...,0.067281,0.548762,0.687274,-0.06768,0.081745,0.545664,0.680934,-0.075489,0.090029,hungry


In [31]:
dataset.to_csv('data/dataset_test_hungry.csv')

# Load Data TESTS

In [35]:
y = pd.read_csv('data/train.csv')
y = y.loc[y.sign=='hungry'].loc[y.participant_id==25571].head(1)
y

,path,participant_id,sequence_id,sign
150,train_landmark_files/25571/1005995721.parquet,25571,1005995721,hungry


In [32]:
def update_dataset(dataset, signs, col_names):
    print(f'recordings found: {y.loc[[s in signs for s in y.sign]].shape[0]}')
    for i, sign in enumerate(signs):
        print(f'sign: {sign}, {i+1}/{len(signs)}')
        for j, participant_id in enumerate(y.participant_id.unique()):
            print(f'    pid: {j+1}/21')
            y_subset = y.loc[(y.sign == sign) & (y.participant_id == participant_id)]
            data = []
            n = y_subset.shape[0]
            for k, path in enumerate(y_subset.path):
                print(f'        recording: {k+1}/{n}')
                path_D = path
                df = pd.read_parquet(path_D)
                this_data = []
                for type in set(df.type):
                    print(type)
                    df_t = df.loc[df.type == type]
                    for lm_i in set(df_t.landmark_index):
                        df_t_i = df_t.loc[(df_t['x'].notna()) & (df_t['y'].notna()) & (df_t.landmark_index == lm_i)]
                        if df_t_i.shape[0] != 0:
                            this_data.append(df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[0]['y'])
                            this_data.append(df_t_i.iloc[-1]['x'] - df_t_i.iloc[0]['x'])
                            this_data.append(df_t_i.iloc[-1]['y'] - df_t_i.iloc[0]['y'])
                        else:
                            this_data.extend([0,0,0,0])
                data.append(this_data + [sign])
            sign_df = pd.DataFrame(data, columns=col_names)
            dataset = pd.concat((dataset,sign_df), axis=0, ignore_index=True)
    return dataset

#### Signs used on round two of data collection
['pig',
 'elephant',
 'mad',
 'finger',
 'green',
 'up',
 'bad',
 'finger',
 'bye',
 'airplane',
 'mom',
 'kitty',
 'home',
 'yellow',
 'shirt',
 'talk',
 'icecream',
 'green',
 'clown',
 'lion',
 'bad',
 'thirsty',
 'elephant',
 'book',
 'shirt',
 'dog',
 'finger',
 'pretty',
 'vacuum',
 'vacuum',
 'chocolate',
 'pretty',
 'same',
 'red',
 'every',
 'feet',
 'drawer',
 'gum',
 'drop',
 'story',
 'puppy',
 'haveto',
 'book',
 'bad',
 'zebra',
 'cat',
 'cow',
 'mad',
 'dryer',
 'old',
 'clown',
 'garbage',
 'stairs',
 'cute',
 'wolf',
 'for',
 'tree',
 'stairs',
 'sick',
 'finish',
 'bird',
 'nose',
 'no',
 'drop',
 'yellow',
 'garbage',
 'have',
 'blow',
 'wet',
 'airplane',
 'vacuum',
 'no',
 'bye',
 'nose',
 'give',
 'puppy',
 'gum',
 'dry',
 'shower',
 'another',
 'every',
 'haveto',
 'wolf',
 'bee',
 'empty',
 'puppy',
 'thirsty',
 'night',
 'gum',
 'sick',
 'drawer',
 'nose',
 'have',
 'pretty',
 'airplane',
 'cry',
 'tiger',
 'mom',
 'talk',
 'have',
 'green',
 'zebra',
 'because',
 'gum',
 'puppy',
 'mad',
 'wolf',
 'donkey',
 'yes',
 'yucky',
 'why',
 'no',
 'tree',
 'tree',
 'dryer',
 'morning',
 'that',
 'orange',
 'where',
 'wolf',
 'tomorrow',
 'tomorrow',
 'gum',
 'uncle',
 'shower',
 'mouth',
 'nose',
 'farm',
 'frog',
 'table',
 'green',
 'another',
 'helicopter',
 'talk',
 'dry',
 'hear',
 'yourself',
 'cute',
 'gum',
 'garbage',
 'dryer',
 'find',
 'kitty',
 'chocolate',
 'no',
 'bird',
 'girl',
 'story',
 'red',
 'cry',
 'story',
 'yourself',
 'vacuum',
 'green',
 'another']

In [33]:
col_names = pd.read_csv('data/col_names.csv', index_col=0)['0']
dataset = pd.DataFrame([],columns=col_names)

dataset = update_dataset(dataset,signs,col_names)


recordings found: 1
sign: hungry, 1/1
    pid: 1/21
        recording: 1/1
left_hand
pose
face
right_hand
